In [1]:
# === E-commerce Joins & Reshaping – Final (Colab-ready) ===
import pandas as pd
import numpy as np

pd.set_option("display.width", 140)
pd.set_option("display.max_rows", 30)

# ----------------------------------------------------------
# 1) Load data (CSV if present; otherwise use a mock dataset)
# ----------------------------------------------------------
CSV_PATH = "/content/data.csv"   # change if you upload elsewhere
df_raw = None

try:
    df_raw = pd.read_csv(CSV_PATH, encoding="ISO-8859-1")
    print("Loaded real CSV:", df_raw.shape)
except Exception as e:
    print("Could not load real CSV, falling back to mock. Error:", e)

if not isinstance(df_raw, pd.DataFrame):
    data = {
        "InvoiceNo": ["10001","10001","10002","10003","10003","10003"],
        "StockCode": ["A1","B2","A1","C3","A1","B2"],
        "Description": ["Mug","T-shirt","Mug","Hat","Mug","T-shirt"],
        "Quantity": [2,1,3,1,1,2],
        "InvoiceDate": [
            "2010-12-01 08:45","2010-12-01 08:45","2010-12-02 10:05",
            "2010-12-05 14:20","2010-12-05 14:20","2010-12-05 14:20"
        ],
        "UnitPrice": [5.0,15.0,5.0,10.0,5.0,15.0],
        "CustomerID": [12345,12345,99999,77777,77777,77777],
        "Country": ["United Kingdom","United Kingdom","France","Germany","Germany","Germany"]
    }
    df_raw = pd.DataFrame(data)
    print("Created mock dataset:", df_raw.shape)

assert isinstance(df_raw, pd.DataFrame), "df_raw is not a DataFrame!"

# ----------------------------------------------------------
# 2) Clean columns & types
# ----------------------------------------------------------
df = df_raw.copy()
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_", regex=False)
)

# Parse datetime; coerce bad rows to NaT (no crash)
df["invoicedate"] = pd.to_datetime(df["invoicedate"], errors="coerce")

# Force numeric types safely
for c in ["quantity", "unitprice"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Drop rows missing critical fields
df = df.dropna(subset=["invoicedate", "quantity", "unitprice", "invoiceno"])

# Optional tidy-up
if "country" in df.columns:
    df["country"] = df["country"].astype(str).str.strip()

print("\nCleaned DF info:")
print(df.info())

# ----------------------------------------------------------
# 3) Split into logical tables (orders vs items)
# ----------------------------------------------------------
orders_cols = ["invoiceno", "customerid", "invoicedate", "country"]
order_items_cols = ["invoiceno", "stockcode", "description", "quantity", "unitprice"]

orders = (
    df[orders_cols]
      .drop_duplicates(subset=["invoiceno"])
      .reset_index(drop=True)
)

order_items = df[order_items_cols].reset_index(drop=True)

print("\nShapes → orders:", orders.shape, "order_items:", order_items.shape)

# ----------------------------------------------------------
# 4) Merge (inner join on InvoiceNo)
# ----------------------------------------------------------
df_full = pd.merge(order_items, orders, on="invoiceno", how="inner")
print("Merged shape:", df_full.shape)

# ----------------------------------------------------------
# 5) Revenue & month features
# ----------------------------------------------------------
df_full["revenue"] = df_full["quantity"] * df_full["unitprice"]
df_full["month"] = df_full["invoicedate"].dt.to_period("M")

print("\nQuick checks:")
print("Total revenue:", float(df_full["revenue"].sum()))
print(df_full[["invoiceno","stockcode","quantity","unitprice","revenue","month"]].head())

# ----------------------------------------------------------
# 6) Monthly revenue by country (pivot) + long (melt)
# ----------------------------------------------------------
pivot_rev = pd.pivot_table(
    df_full,
    values="revenue",
    index="month",
    columns="country",
    aggfunc="sum",
    margins=True,
    margins_name="Total"
).fillna(0).round(2)

print("\nPivot (monthly revenue by country):")
print(pivot_rev.head(12))

melted = (
    pivot_rev.reset_index()
             .melt(id_vars="month", var_name="country", value_name="revenue")
)
melted = melted[melted["country"] != "Total"].reset_index(drop=True)

print("\nMelted (long form):")
print(melted.head(10))

# ----------------------------------------------------------
# 7) Rank: Top-3 countries by revenue per month (reusable)
# ----------------------------------------------------------
def top_k_countries_by_month(df_full: pd.DataFrame, k: int = 3) -> pd.DataFrame:
    g = df_full.groupby(["month", "country"], as_index=False)["revenue"].sum()
    s = g.sort_values(["month", "revenue"], ascending=[True, False])
    return s.groupby("month", group_keys=False).head(k)

top_countries = top_k_countries_by_month(df_full, k=3)
print("\nTop-3 countries per month:")
print(top_countries)

# ----------------------------------------------------------
# 8) Sanity checks (totals & non-negativity)
# ----------------------------------------------------------
total_raw = df_full["revenue"].sum()
total_grouped = df_full.groupby(["month", "country"])["revenue"].sum().sum()

assert np.isclose(total_raw, total_grouped), "Mismatch in totals between raw and grouped!"
assert (df_full["revenue"] >= 0).all(), "Negative revenue found (returns/credits?)."

print("\n✅ Sanity checks passed.")

# ----------------------------------------------------------
# 9) Exports
# ----------------------------------------------------------
pivot_rev.to_csv("monthly_revenue_by_country.csv")
melted.to_csv("monthly_revenue_long.csv", index=False)
top_countries.to_csv("top3_countries_per_month.csv", index=False)

print("\n📦 Exports written:")
print(" - monthly_revenue_by_country.csv")
print(" - monthly_revenue_long.csv")
print(" - top3_countries_per_month.csv")

Could not load real CSV, falling back to mock. Error: [Errno 2] No such file or directory: '/content/data.csv'
Created mock dataset: (6, 8)

Cleaned DF info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   invoiceno    6 non-null      object        
 1   stockcode    6 non-null      object        
 2   description  6 non-null      object        
 3   quantity     6 non-null      int64         
 4   invoicedate  6 non-null      datetime64[ns]
 5   unitprice    6 non-null      float64       
 6   customerid   6 non-null      int64         
 7   country      6 non-null      object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 516.0+ bytes
None

Shapes → orders: (3, 4) order_items: (6, 5)
Merged shape: (6, 8)

Quick checks:
Total revenue: 85.0
  invoiceno stockcode  quantity  unitprice  revenue    month